In [1]:
! pip install --quiet pandas numpy

In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 50)

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/rjribeiro/machine_learning/main/naive_bayes/NB_CarSell2_Data2.csv")
X = df.iloc[:, :3]
y = df.target

In [4]:
df

,price,lug_boot,safety,target
0,low,med,med,acc
1,low,med,high,acc
2,low,small,med,acc
3,med,small,med,acc
4,med,big,high,acc
5,high,small,high,acc
6,low,big,high,acc
7,high,med,med,acc
8,med,med,med,acc
9,med,big,med,acc


# Afirmação 1

A priori, isto é,  sem considerar os atributos de cada instância, é menos provável que uma nova instância seja da classe acc do que da classe unacc.


In [5]:
def get_priori_probabilities(y: np.ndarray) -> dict[str, float]:
    priori_probabilities = {}
    for target in np.unique(y.values):
        p_target = (y == target).mean()
        priori_probabilities[target] = p_target

    return priori_probabilities

In [6]:
priori_probabilities = get_priori_probabilities(y)
priori_probabilities

{'acc': 0.4, 'unacc': 0.6}

**Correto**

P(acc) = 0.4
P(unacc) = 0.6

# Afirmação 2


O modelo Naïve Bayes considera que os atributos price, lug_boot e safety são estatisticamente independentes uns dos outros condicionado ao valor da classe (acc ou unacc).

**Correto**

Essa suposição é para facilitar o cálculo, visto que com os atributos estaticamente dependentes
esse cálculo poderia ter um custo computacional muito grande.

# Afirmação 3

A probabilidade condicional do atributo lug_boot igual à "med" é maior para a classe acc do que para a classe unacc. Isto é, P(lug_boot=med|target=acc) > P(lug_boot=med|target=unacc).

In [7]:
(df[df['target'] == 'acc']['lug_boot'] == 'med').mean()

0.4

In [8]:
(df[df['target'] == 'unacc']['lug_boot'] == 'med').mean()

0.26666666666666666

**Correto**

P(lug_boot=med|target=acc) = 0.4
P(lug_boot=med|target=unacc) = 0.26

# Afirmação 4

Nenhuma instância com o atributo safety = low seria predita como acc (aceitável) por este classificador pelo problema da probabilidade zero. Isto é, com base nestes dados de treinamento, P(safety=low|target=acc) = 0.

In [9]:
(df[df['target'] == 'acc']['safety'] == 'low').mean()

0.0

**Correto**

P(safety=low|target=acc) = 0

# Afirmação 5


As estimativas das probabilidades a posteriori para uma nova instância x com atributos price=high, lug_boot=med e safety=med seriam aproximadamente P(target=acc|x) = 0.019  e  P(target=unacc|x) = 0.021. Portanto, esta instância será classificada como unacc pelo modelo.

In [10]:
prod_acc = 1
prod_unacc = 1

In [11]:
p_price_high_acc = (df[df['target'] == 'acc']['price'] == 'high').mean()
prod_acc *= p_price_high_acc
p_price_high_acc

0.2

In [12]:
p_price_high_unacc = (df[df['target'] == 'unacc']['price'] == 'high').mean()
prod_unacc *= p_price_high_unacc
p_price_high_unacc

0.3333333333333333

In [13]:
p_lugboot_med_acc = (df[df['target'] == 'acc']['lug_boot'] == 'med').mean()
prod_acc *= p_lugboot_med_acc
p_lugboot_med_acc

0.4

In [14]:
p_lugboot_med_unacc = (df[df['target'] == 'unacc']['lug_boot'] == 'med').mean()
prod_unacc *= p_lugboot_med_unacc
p_lugboot_med_unacc

0.26666666666666666

In [15]:
p_safety_med_acc = (df[df['target'] == 'acc']['safety'] == 'med').mean()
prod_acc *= p_safety_med_acc
p_safety_med_acc

0.6

In [16]:
p_safety_med_unacc = (df[df['target'] == 'unacc']['safety'] == 'med').mean()
prod_acc *= p_safety_med_unacc
p_safety_med_unacc

0.4

In [17]:
prod_acc * priori_probabilities["acc"]

0.007680000000000003

In [18]:
prod_unacc * priori_probabilities["unacc"]

0.05333333333333332

Incorreto

A classe de maior probabilidade a posteriori é **unacc**, porém com P(unacc | x) = 0.053 e P(acc | x) = 0.0077